In [1]:
from mmvae_hub.base.experiment_vis.utils import *
from mmvae_hub.base.utils.MongoDB import MongoDatabase
import os
from pathlib import Path

[13:52:36 CEST] Log file: /usr/local/var/logs/mmvae_hub/UTC-20210509-115236.log


In [ ]:
exp_db = MongoDatabase(training=False)
experiments = exp_db.connect()
